In [3]:
# 1. Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import glob
import os

# 2. Constants and Configuration
dataset_url = r'C:\Users\abudh\Desktop\CropWatch\EuroSAT\2750'
model_path = r'C:\Users\abudh\Desktop\CropWatch\Training_model.h5'

batch_size = 64
img_height = 64
img_width = 64
validation_split = 0.2
no_epochs = 25

# 3. Data Augmentation and Loading
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.2,
    fill_mode='nearest',
    validation_split=validation_split
)

dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset_url, image_size=(img_height, img_width), batch_size=batch_size)

train_dataset = datagen.flow_from_directory(
    directory=dataset_url,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="training",
    #shuffle=False,
    #seed=0,
    class_mode='categorical'
)

val_dataset = datagen.flow_from_directory(
    directory=dataset_url,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="validation",
    #shuffle=False,
    #seed=0,
    class_mode='categorical'
)

# 4. Class Weights Calculation
class_indices = train_dataset.class_indices
class_labels = list(class_indices.keys())
class_indices = list(class_indices.values())

class_counts_dict = {class_name: len(glob.glob(os.path.join(dataset_url, class_name, '*'))) for class_name in class_labels}
class_counts = np.array([class_counts_dict[class_name] for class_name in class_labels])

weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(class_indices),
    y=np.array([train_dataset.classes[i] for i in range(len(train_dataset.classes))])
)

class_weight_dict = dict(zip(class_labels, weights))
print("Class weights:")
for label, weight in class_weight_dict.items():
    print(f"Class '{label}': {weight:.4f}")

# 5. Model Definition
def identity_block(X, f, filters, training=True, initializer=tf.keras.initializers.RandomUniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(F1, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    return X

def convolutional_block(X, f, filters, s=2, training=True, initializer=tf.keras.initializers.GlorotUniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(F1, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_labels)):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = AveragePooling2D(pool_size=(2, 2))(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=X)
    return model

# 6. Callbacks
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=15,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    verbose=1,
    restore_best_weights=True
)

# 7. Model Compilation and Training
model = ResNet50()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    epochs=no_epochs,
    validation_data=val_dataset,
    class_weight=class_weight_dict,  # Add this line
    callbacks=[reduce_lr, early_stopping]
)

Found 27000 files belonging to 10 classes.
Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.
Class weights:
Class 'AnnualCrop': 0.9000
Class 'Forest': 0.9000
Class 'HerbaceousVegetation': 0.9000
Class 'Highway': 1.0800
Class 'Industrial': 1.0800
Class 'Pasture': 1.3500
Class 'PermanentCrop': 1.0800
Class 'Residential': 0.9000
Class 'River': 1.0800
Class 'SeaLake': 0.9000
Epoch 1/25


C:\Users\abudh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


338/338 ━━━━━━━━━━━━━━━━━━━━ 664s 2s/step - accuracy: 0.1308 - loss: 2.7137 - val_accuracy: 0.0926 - val_loss: 2.5225 - learning_rate: 1.0000e-04
Epoch 2/25
338/338 ━━━━━━━━━━━━━━━━━━━━ 600s 2s/step - accuracy: 0.2496 - loss: 2.1398 - val_accuracy: 0.0839 - val_loss: 2.7004 - learning_rate: 1.0000e-04
Epoch 3/25
315/338 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.5223 - loss: 1.4414

KeyboardInterrupt: 

In [ ]:
# 8. Model Evaluation and Saving
# Save model in native Keras format
model.save('Training_Model200.keras')
model_path = r'C:\Users\abudh\Desktop\CropWatch\Training_model555.keras'
model.save(model_path)

# Load the model
loaded_model = load_model('Training_Model200.keras')

# Evaluate the model
loss, accuracy = loaded_model.evaluate(val_dataset)
print(f"Test Accuracy: {accuracy:.4f}")

# 9. Visualization

# Sample Images
class_names = list(train_dataset.class_indices.keys())
plt.figure(figsize=(10, 10))
for images, labels in train_dataset:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[np.argmax(labels[i])])  # No need for .numpy()
        plt.axis("off")
    break  # Exit loop after displaying the first batch
plt.show()

# Training History
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend(['Train', 'Validation'])
plt.show()

# Confusion Matrix
y_true = []
y_pred = []

# Extract all true labels and predictions from the validation dataset
for images, labels in val_dataset:
    predictions = loaded_model.predict(images)
    y_true.extend(np.argmax(labels, axis=1))  # No need for .numpy()
    y_pred.extend(np.argmax(predictions, axis=1))

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
loaded_model = load_model('Training_Model200.keras')

# Training History
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend(['Train', 'Validation'])
plt.show()